In [33]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.utils import shuffle

# Load your dataset
df = pd.read_csv('PAT_Dataset.csv')

# Shuffle the dataset
df = shuffle(df, random_state=42)

# Columns containing the target variables
target_columns = [
    'steady_state_temp_L0', 'steady_state_temp_L1', 'router_avg_temp_L0', 'router_avg_temp_L1',
    'core_avg_temp_L0', 'core_avg_temp_L1', 'mem_avg_temp_L0', 'mem_avg_temp_L1',
    'total_area', 'avg_power', 'avg_cores_power', 'avg_routers_power', 'avg_power_per_router',
    'layer_area', 'area_per_core'
]

# Identify categorical columns
categorical_columns = ['routing_type', 'selection_strategy', 'traffic_type']

# One-hot encode categorical columns
X = pd.get_dummies(df.drop(columns=target_columns), columns=categorical_columns, drop_first=True)
y = df[target_columns]

# Split the data into training and test sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize StandardScaler
scaler_X = StandardScaler()
scaler_y = StandardScaler()

# Scale the feature and target data
X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)
y_train_scaled = scaler_y.fit_transform(y_train)
y_test_scaled = scaler_y.transform(y_test)


In [34]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Reshape the data to fit a CNN input
# The CNN model expects a 3D input shape (samples, time steps, features), so we reshape X_train_scaled and X_test_scaled
X_train_cnn = X_train_scaled.reshape(X_train_scaled.shape[0], X_train_scaled.shape[1], 1)  # Add 1 for the single "channel"
X_test_cnn = X_test_scaled.reshape(X_test_scaled.shape[0], X_test_scaled.shape[1], 1)

# Build a simple CNN model
cnn_model = Sequential()

# Add 1D convolutional layers with padding='same' to avoid shrinking the input size
cnn_model.add(Conv1D(filters=64, kernel_size=3, activation='relu', padding='same', input_shape=(X_train_cnn.shape[1], 1)))
cnn_model.add(MaxPooling1D(pool_size=2))

cnn_model.add(Conv1D(filters=128, kernel_size=3, activation='relu', padding='same'))
cnn_model.add(MaxPooling1D(pool_size=2))

# Flatten the output of the convolutions
cnn_model.add(Flatten())

# Add dense layers for regression
cnn_model.add(Dense(128, activation='relu'))
cnn_model.add(Dense(64, activation='relu'))
cnn_model.add(Dense(len(target_columns)))  # Output layer for all targets at once

# Compile the CNN model
cnn_model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
cnn_model.fit(X_train_cnn, y_train_scaled, epochs=10, batch_size=32, validation_split=0.2)

# Predict using the trained CNN model
y_pred_cnn_scaled = cnn_model.predict(X_test_cnn)

# Prepare a list to store the evaluation results
cnn_evaluation_results = []

# Evaluate the model for each target variable separately
print("CNN Performance (per target variable):")
for i, target in enumerate(target_columns):
    y_test_target = y_test_scaled[:, i]  # Get actual values for target i
    y_pred_target = y_pred_cnn_scaled[:, i]  # Get predicted values for target i

    # Calculate performance metrics
    mse = mean_squared_error(y_test_target, y_pred_target)
    mae = mean_absolute_error(y_test_target, y_pred_target)
    r2 = r2_score(y_test_target, y_pred_target)

    # Append results to the list
    cnn_evaluation_results.append({
        'algorithm': 'CNN',
        'target': target,
        'MSE': mse,
        'MAE': mae,
        'R²': r2
    })

    # Print results for the current target
    print(f"\nTarget: {target}")
    print("MSE:", mse)
    print("MAE:", mae)
    print("R²:", r2)
    print("-" * 30)

# Convert the results list to a DataFrame
cnn_evaluation_df = pd.DataFrame(cnn_evaluation_results)

# Display the CNN evaluation results DataFrame
print("\nCNN Evaluation Results DataFrame:")
print(cnn_evaluation_df)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


516/516 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 0.4400 - val_loss: 0.1242
Epoch 2/10
516/516 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.1163 - val_loss: 0.0816
Epoch 3/10
516/516 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0808 - val_loss: 0.0680
Epoch 4/10
516/516 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.0600 - val_loss: 0.0433
Epoch 5/10
516/516 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.0419 - val_loss: 0.0337
Epoch 6/10
516/516 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0326 - val_loss: 0.0260
Epoch 7/10
516/516 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0236 - val_loss: 0.0167
Epoch 8/10
516/516 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0161 - val_loss: 0.0149
Epoch 9/10
516/516 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0137 - val_loss: 0.0129
Epoch 10/10
516/516 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0142 - val_loss: 0.0102
162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
CNN Performance (per target variable):

Target: steady_state_temp_L0
MSE: 0.02096937110308532
MAE: 0.08261156

In [35]:
import pandas as pd

# Define the new data as a DataFrame
new_data = pd.DataFrame({
    'dimx': [14],
    'dimy': [11],
    'dimz': [2],
    'buffer_size': [10],
    'packet_size_min': [4],
    'packet_size_max': [8],
    'routing_type_oe_3d': [1],  # Make sure to align with the encoded columns
    'selection_strategy_thermal': [1],
    'traffic_type_random': [1],
    'injection_rate': [0.05]
})

# Ensure all columns are present (including any missing dummy variables from the training data)
# Missing columns in new_data are filled with 0s
for col in X.columns:
    if col not in new_data:
        new_data[col] = 0

# Reorder the new_data columns to match the original feature order
new_data = new_data[X.columns]

# Scale the new data using the previously fitted scaler
new_data_scaled = scaler_X.transform(new_data)

# Reshape the scaled data to fit the CNN input shape
new_data_cnn = new_data_scaled.reshape(1, new_data_scaled.shape[1], 1)  # Shape (1, num_features, 1)

# Make predictions with the CNN model
new_data_pred_scaled = cnn_model.predict(new_data_cnn)

# Inverse scale the predictions to get original scale
new_data_pred = scaler_y.inverse_transform(new_data_pred_scaled)

# Convert the prediction to a DataFrame for readability
predictions_df = pd.DataFrame(new_data_pred, columns=target_columns)

# Display the predictions
print("Predictions for the new data:")
print(predictions_df)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
Predictions for the new data:
   steady_state_temp_L0  steady_state_temp_L1  router_avg_temp_L0  \
0            143.108994            141.062057           26.920168   

   router_avg_temp_L1  core_avg_temp_L0  core_avg_temp_L1  mem_avg_temp_L0  \
0           26.680582         26.157522         26.034374        25.874044   

   mem_avg_temp_L1   total_area     avg_power  avg_cores_power  \
0         25.76001  718245888.0  4.409188e-07     3.453331e-07   

   avg_routers_power  avg_power_per_router   layer_area  area_per_core  
0       8.806025e-08          2.926545e-10  368154816.0      4695230.0  


In [36]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import Dropout

# Build an improved CNN model with dropout layers and deeper architecture
cnn_model = Sequential()

cnn_model.add(Conv1D(filters=64, kernel_size=3, activation='relu', padding='same', input_shape=(X_train_cnn.shape[1], 1)))
cnn_model.add(MaxPooling1D(pool_size=2))
cnn_model.add(Dropout(0.3))

cnn_model.add(Conv1D(filters=128, kernel_size=3, activation='relu', padding='same'))
cnn_model.add(MaxPooling1D(pool_size=2))
cnn_model.add(Dropout(0.3))

cnn_model.add(Conv1D(filters=256, kernel_size=3, activation='relu', padding='same'))
cnn_model.add(MaxPooling1D(pool_size=2))
cnn_model.add(Dropout(0.3))

# Flatten and add dense layers for regression
cnn_model.add(Flatten())
cnn_model.add(Dense(128, activation='relu'))
cnn_model.add(Dropout(0.3))
cnn_model.add(Dense(64, activation='relu'))
cnn_model.add(Dense(len(target_columns)))  # Output layer for all targets at once

# Compile the model
cnn_model.compile(optimizer='adam', loss='mean_squared_error')

# Set up callbacks for early stopping and learning rate reduction
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-5)

# Train the model
cnn_model.fit(X_train_cnn, y_train_scaled, epochs=50, batch_size=32, validation_split=0.2,
              callbacks=[early_stopping, reduce_lr])

# Predict using the trained CNN model
y_pred_cnn_scaled = cnn_model.predict(X_test_cnn)

# Prepare evaluation results as before
cnn_evaluation_results = []
print("CNN Performance (per target variable):")

for i, target in enumerate(target_columns):
    y_test_target = y_test_scaled[:, i]
    y_pred_target = y_pred_cnn_scaled[:, i]
    mse = mean_squared_error(y_test_target, y_pred_target)
    mae = mean_absolute_error(y_test_target, y_pred_target)
    r2 = r2_score(y_test_target, y_pred_target)
    cnn_evaluation_results.append({
        'algorithm': 'CNN',
        'target': target,
        'MSE': mse,
        'MAE': mae,
        'R²': r2
    })
    print(f"\nTarget: {target}")
    print("MSE:", mse)
    print("MAE:", mae)
    print("R²:", r2)
    print("-" * 30)

# Results DataFrame
cnn_evaluation_df = pd.DataFrame(cnn_evaluation_results)
print("\nCNN Evaluation Results DataFrame:")
print(cnn_evaluation_df)


Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


516/516 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - loss: 0.5703 - val_loss: 0.1989 - learning_rate: 0.0010
Epoch 2/50
516/516 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.2484 - val_loss: 0.1069 - learning_rate: 0.0010
Epoch 3/50
516/516 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 0.1773 - val_loss: 0.0833 - learning_rate: 0.0010
Epoch 4/50
516/516 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.1434 - val_loss: 0.0801 - learning_rate: 0.0010
Epoch 5/50
516/516 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.1329 - val_loss: 0.0571 - learning_rate: 0.0010
Epoch 6/50
516/516 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 0.1214 - val_loss: 0.0584 - learning_rate: 0.0010
Epoch 7/50
516/516 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.1135 - val_loss: 0.0616 - learning_rate: 0.0010
Epoch 8/50
516/516 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.1039 - val_loss: 0.0426 - learning_rate: 0.0010
Epoch 9/50
516/516 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0964 - val_loss: 0.0446 - learning_rate: 0.0010
Epoch 10/50
516/516 ━

In [37]:
import pandas as pd

new_data = pd.DataFrame({
    'dimx': [14],
    'dimy': [11],
    'dimz': [2],
    'buffer_size': [10],
    'packet_size_min': [4],
    'packet_size_max': [8],
    'routing_type_oe_3d': [1],  # Make sure to align with the encoded columns
    'selection_strategy_thermal': [1],
    'traffic_type_random': [1],
    'injection_rate': [0.05]
})

# Ensure all columns are present (including any missing dummy variables from the training data)
# Missing columns in new_data are filled with 0s
for col in X.columns:
    if col not in new_data:
        new_data[col] = 0

# Reorder the new_data columns to match the original feature order
new_data = new_data[X.columns]

# Scale the new data using the previously fitted scaler
new_data_scaled = scaler_X.transform(new_data)

# Reshape the scaled data to fit the CNN input shape
new_data_cnn = new_data_scaled.reshape(1, new_data_scaled.shape[1], 1)  # Shape (1, num_features, 1)

# Make predictions with the CNN model
new_data_pred_scaled = cnn_model.predict(new_data_cnn)

# Inverse scale the predictions to get original scale
new_data_pred = scaler_y.inverse_transform(new_data_pred_scaled)

# Convert the prediction to a DataFrame for readability
predictions_df = pd.DataFrame(new_data_pred, columns=target_columns)

# Display the predictions
print("Predictions for the new data:")
print(predictions_df)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
Predictions for the new data:
   steady_state_temp_L0  steady_state_temp_L1  router_avg_temp_L0  \
0            141.590027            140.825592           26.857075   

   router_avg_temp_L1  core_avg_temp_L0  core_avg_temp_L1  mem_avg_temp_L0  \
0           26.589748         26.159397         25.996588        25.862862   

   mem_avg_temp_L1   total_area     avg_power  avg_cores_power  \
0        25.731989  729290496.0  4.319029e-07     3.490335e-07   

   avg_routers_power  avg_power_per_router   layer_area  area_per_core  
0       8.383054e-08          2.812783e-10  365363648.0      4695230.0  


In [40]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Reshape the data to fit an RNN input (samples, time steps, features)
X_train_rnn = X_train_scaled.reshape(X_train_scaled.shape[0], X_train_scaled.shape[1], 1)  # 1 channel for each time step
X_test_rnn = X_test_scaled.reshape(X_test_scaled.shape[0], X_test_scaled.shape[1], 1)  # Same reshaping for test set

# Build a deeper RNN model using multiple LSTM layers
rnn_model = Sequential()

# Add LSTM layers with more units and dropout for regularization
rnn_model.add(LSTM(128, activation='relu', input_shape=(X_train_rnn.shape[1], 1), return_sequences=True))
rnn_model.add(Dropout(0.2))  # Add dropout to prevent overfitting

rnn_model.add(LSTM(256, activation='relu', return_sequences=True))  # Deeper layer
rnn_model.add(Dropout(0.2))  # Add dropout to prevent overfitting

rnn_model.add(LSTM(512, activation='relu', return_sequences=False))  # Even deeper LSTM layer
rnn_model.add(Dropout(0.3))  # Higher dropout rate

# Add dense layers for regression
rnn_model.add(Dense(256, activation='relu'))  # Larger dense layer
rnn_model.add(Dense(128, activation='relu'))  # Another dense layer
rnn_model.add(Dense(64, activation='relu'))   # Another dense layer
rnn_model.add(Dense(len(target_columns)))  # Output layer for all targets at once

# Compile the RNN model
rnn_model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
rnn_model.fit(X_train_rnn, y_train_scaled, epochs=20, batch_size=32, validation_split=0.2)

# Predict using the trained RNN model
y_pred_rnn_scaled = rnn_model.predict(X_test_rnn)

# Prepare a list to store the evaluation results
rnn_evaluation_results = []

# Evaluate the model for each target variable separately
print("Deeper RNN (LSTM) Performance (per target variable):")
for i, target in enumerate(target_columns):
    y_test_target = y_test_scaled[:, i]  # Get actual values for target i
    y_pred_target = y_pred_rnn_scaled[:, i]  # Get predicted values for target i

    # Calculate performance metrics
    mse = mean_squared_error(y_test_target, y_pred_target)
    mae = mean_absolute_error(y_test_target, y_pred_target)
    r2 = r2_score(y_test_target, y_pred_target)

    # Append results to the list
    rnn_evaluation_results.append({
        'algorithm': 'Deeper RNN (LSTM)',
        'target': target,
        'MSE': mse,
        'MAE': mae,
        'R²': r2
    })

    # Print results for the current target
    print(f"\nTarget: {target}")
    print("MSE:", mse)
    print("MAE:", mae)
    print("R²:", r2)
    print("-" * 30)

# Convert the results list to a DataFrame
rnn_evaluation_df = pd.DataFrame(rnn_evaluation_results)

# Display the RNN evaluation results DataFrame
print("\nDeeper RNN Evaluation Results DataFrame:")
print(rnn_evaluation_df)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/20
516/516 ━━━━━━━━━━━━━━━━━━━━ 15s 12ms/step - loss: 0.6645 - val_loss: 0.5664
Epoch 2/20
516/516 ━━━━━━━━━━━━━━━━━━━━ 15s 10ms/step - loss: 0.5028 - val_loss: 0.4097
Epoch 3/20
516/516 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - loss: 0.3652 - val_loss: 0.3071
Epoch 4/20
516/516 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - loss: 0.2490 - val_loss: 0.1026
Epoch 5/20
516/516 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - loss: 0.1034 - val_loss: 0.0504
Epoch 6/20
516/516 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - loss: 0.0703 - val_loss: 0.0983
Epoch 7/20
516/516 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - loss: 0.0864 - val_loss: 0.0688
Epoch 8/20
516/516 ━━━━━━━━━━━━━━━━━━━━ 10s 10ms/step - loss: 0.0663 - val_loss: 0.0313
Epoch 9/20
516/516 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - loss: 0.0487 - val_loss: 0.0272
Epoch 10/20
516/516 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - loss: 0.0400 - val_loss: 0.0256
Epoch 11/20
516/516 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - loss: 0.0424 - val_loss: 0.0534
Epoch 12/20
516/516 ━━━━━━━━━━━━━━━━━

In [42]:
import pandas as pd

new_data = pd.DataFrame({
    'dimx': [3],
    'dimy': [10],
    'dimz': [2],
    'buffer_size': [8],
    'packet_size_min': [4],
    'packet_size_max': [8],
    'routing_type_fullyadaptive': [1],    # Make sure to align with the encoded columns
    'selection_strategy_thermal': [1],
    'traffic_type_random': [1],
    'injection_rate': [0.06]
})

# Ensure all columns are present (including any missing dummy variables from the training data)
# Missing columns in new_data are filled with 0s
for col in X.columns:
    if col not in new_data:
        new_data[col] = 0

# Reorder the new_data columns to match the original feature order
new_data = new_data[X.columns]

# Scale the new data using the previously fitted scaler
new_data_scaled = scaler_X.transform(new_data)

# Reshape the scaled data to fit the CNN input shape
new_data_cnn = new_data_scaled.reshape(1, new_data_scaled.shape[1], 1)  # Shape (1, num_features, 1)

# Make predictions with the CNN model
new_data_pred_scaled = rnn_model.predict(new_data_cnn)

# Inverse scale the predictions to get original scale
new_data_pred = scaler_y.inverse_transform(new_data_pred_scaled)

# Convert the prediction to a DataFrame for readability
predictions_df = pd.DataFrame(new_data_pred, columns=target_columns)

# Display the predictions
print("Predictions for the new data:")
print(predictions_df)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Predictions for the new data:
   steady_state_temp_L0  steady_state_temp_L1  router_avg_temp_L0  \
0             67.197083             67.573349           25.606779   

   router_avg_temp_L1  core_avg_temp_L0  core_avg_temp_L1  mem_avg_temp_L0  \
0           25.979721         25.475954         25.574236         25.37184   

   mem_avg_temp_L1   total_area     avg_power  avg_cores_power  \
0        25.447403  160086704.0  8.112868e-08     7.021715e-08   

   avg_routers_power  avg_power_per_router  layer_area  area_per_core  
0       1.132547e-08          1.961568e-10  75313344.0      4695230.0  
